# Typeclass resolution

Follows the description of the algorithms presented in the paper [**Tabled Typeclass Resolution**](https://arxiv.org/pdf/2001.04301) by Daniel Selsam, Sebastian Ullrich, Leonardo de Moura.

## Class definitions

Some basic definitions of types and typeclasses to demonstrate typeclass resolution on.

In [25]:
from __future__ import annotations
from abc import ABC, abstractmethod
from dataclasses import dataclass
from typing import ClassVar

@dataclass
class Type(ABC):
    @abstractmethod
    def replace_param_indices(self, replace_with: list[Type]) -> Type: ...

@dataclass
class TypeVar(Type):
    index: int
    _current_index: ClassVar[int] = 0

    @staticmethod
    def get_new() -> TypeVar:
        index = TypeVar._current_index
        TypeVar._current_index += 1

        return TypeVar(index=index)
    
    def __str__(self) -> str:
        return "?x_" + str(self.index)

    def replace_param_indices(self, replace_with: list[Type]) -> Type:
        return self

@dataclass
class TypeParamIndex(Type):
    index: int

    def __str__(self) -> str:
        return str(self.index)
    
    def replace_param_indices(self, replace_with: list[Type]) -> Type:
        return replace_with[self.index]

@dataclass
class TypeNameAndArgs(Type):
    name: str
    args: list[Type]

    def __str__(self) -> str:
        return self.name + "".join(" " + str(arg) for arg in self.args)
    
    def replace_param_indices(self, replace_with: list[Type]) -> Type:
        return TypeNameAndArgs(
            name=self.name,
            args=[arg.replace_param_indices(replace_with) for arg in self.args]
        )

@dataclass
class InstanceParam:
    typeclass: str
    args: list[Type]

    def __str__(self) -> str:
        return self.typeclass + " " + " ".join(str(arg) for arg in self.args)
    
    def replace_param_indices(self, replace_with: list[Type]) -> InstanceParam:
        return InstanceParam(
            typeclass=self.typeclass,
            args=[arg.replace_param_indices(replace_with) for arg in self.args]
        )

@dataclass
class TypeclassInstance:
    # Parameters of the instance
    type_params: int
    instance_params: list[InstanceParam]

    # Instance
    typeclass: str
    args: list[Type]

@dataclass
class LookupTable:
    name_to_instance: dict[str, TypeclassInstance]

    def instances_for_typeclass(self, typeclass: str) -> list[str]:
        return [instance_name for instance_name, instance in self.name_to_instance.items() if instance.typeclass == typeclass]

def display_resolution(resolution: list[tuple[InstanceParam, str, list[Type]]], lookup_table: LookupTable):
    for res in resolution:
        print(res[0], "->", str(res[1]), "(inst. of " + lookup_table.name_to_instance[res[1]].typeclass + "".join(" " + str(arg) for arg in res[2]) + ")")

## Basic unification table for type vars

In [26]:
@dataclass
class UnificationTable:
    parent: UnificationTable | None
    table: dict[int, Type]

    def try_resolve(self, ty: Type) -> Type:
        if isinstance(ty, TypeVar):
            if ty.index in self.table:
                return self.table[ty.index]
            
            if self.parent is not None:
                return self.parent.try_resolve(ty)
            
        return ty

    def replace_typevars(self, ty: Type) -> Type:
        if isinstance(ty, TypeVar):
            resolved = self.try_resolve(ty)
            if isinstance(resolved, TypeVar):
                raise Exception("Could not replace typevar " + str(resolved))
            
            return resolved
        if isinstance(ty, TypeNameAndArgs):
            return TypeNameAndArgs(
                name=ty.name,
                args=[self.replace_typevars(arg) for arg in ty.args]
            )
        
        # Unreachable
        raise NotImplementedError()

    def replace_typevars_in_instance_param(self, inst: InstanceParam) -> InstanceParam:
        return InstanceParam(
            typeclass=inst.typeclass,
            args=[self.replace_typevars(arg) for arg in inst.args]
        )

    def get_child(self) -> UnificationTable:
        return UnificationTable(parent=self, table=dict())

    def unify(self, a: Type, b: Type) -> bool:
        a = self.try_resolve(a)
        b = self.try_resolve(b)

        if isinstance(a, TypeVar):
            self.table[a.index] = b

            return True
        elif isinstance(b, TypeVar):
            self.table[b.index] = a

            return True
        elif isinstance(a, TypeNameAndArgs) and isinstance(b, TypeNameAndArgs):
            if a.name != b.name:
                return False
            
            for (a_arg, b_arg) in zip(a.args, b.args):
                if not self.unify(a_arg, b_arg):
                    return False

            return True

        # Unreachable
        raise NotImplementedError()

## SLD based resolution

Typeclass resolution algorithm based on selective linear definite clause (SLD) resolution

*Kowalski, R.: Predicate logic as programming language. In: IFIP congress. vol. 74,
pp. 569–544 (1974)*

In [27]:
def sld_resolve(query: InstanceParam, lookup_table: LookupTable) -> list[tuple[InstanceParam, str, list[Type]]]:
    """Returns a list of typeclass instance parameters and their resolved value. Raises an exception if it's not possible. May not terminate and is prone to exponential blowup."""

    if any(isinstance(arg, TypeParamIndex) for arg in query.args):
        raise Exception("Query cannot contain parameter indices")

    @dataclass
    class Node:
        unresolved_goals: list[InstanceParam]
        resolved_goals: list[tuple[InstanceParam, str, list[Type]]]
        instances_to_try: list[str]
        unification_table: UnificationTable

    # Maintain a stack of nodes. We only test instances against the first unresolved goal. If it succeeds, we'll add a new node with the new goals created by the instance + the remaining goals
    nodes = [Node(
        unresolved_goals=[query], 
        resolved_goals=[], 
        instances_to_try=lookup_table.instances_for_typeclass(query.typeclass), 
        unification_table=UnificationTable(parent=None, table=dict()),
    )]

    while len(nodes) != 0:
        if len(nodes) > 100:
            # We cap out the search at 100 to prevent it from going indefinitely during cycles
            raise Exception("Maximum depth exceeded (probably stuck in a cycle)")

        node = nodes[-1]
        
        if len(node.instances_to_try) == 0:
            # No more instances to try on this node? We can't make progress with it. Go back one step (pop the node) and try something else.
            nodes.pop()
            continue

        # Try to resolve an instance we haven't tried yet against the current goal
        instance_name = node.instances_to_try.pop()

        instance = lookup_table.name_to_instance[instance_name]
        goal = node.unresolved_goals[0]

        # Create typevars for all the instance type params
        typevars: list[Type] = [TypeVar.get_new() for _ in range(instance.type_params)]

        # Try to unify the args of the goal against the args of the instance, in a child table so as to not affect the parent in case we need to backtrack
        table = node.unification_table.get_child()

        instance_args = [instance_arg.replace_param_indices(typevars) for instance_arg in instance.args]

        failure = False
        for (goal_arg, instance_arg) in zip(goal.args, instance_args):
            if not table.unify(goal_arg, instance_arg):
                failure = True
        if failure:
            continue

        # Unification succeeded, proceed with the new goals from the instance + remaining goals of the current node
        new_goals = [instance_param.replace_param_indices(typevars) for instance_param in instance.instance_params] + node.unresolved_goals[1:]
        resolved_goals = node.resolved_goals + [(goal, instance_name, instance_args)]

        if len(new_goals) == 0:
            # No unresolved goals? We got our solution
            return [(node.unification_table.replace_typevars_in_instance_param(goal), val, [table.replace_typevars(arg) for arg in args]) for goal, val, args in resolved_goals]

        nodes.append(Node(
            unresolved_goals=new_goals,
            resolved_goals=resolved_goals,
            instances_to_try=lookup_table.instances_for_typeclass(new_goals[0].typeclass),
            unification_table=table
        ))

        pass

    raise Exception("Failed to resolve query")

### Examples

In [28]:
lookup_table = LookupTable(name_to_instance={
    # Instance that says elements of type Nat can be added together
    "addNat": TypeclassInstance(type_params=0, instance_params=[], typeclass="Add", args=[TypeNameAndArgs(name="Nat", args=[])]),
    # Instance that says elements of type Vec2D that contain things that can be added together can be added together
    "addVec2DAdd": TypeclassInstance(type_params=1, instance_params=[InstanceParam(typeclass="Add", args=[TypeParamIndex(index=0)])], typeclass="Add", args=[TypeNameAndArgs(name="Vec2D", args=[TypeParamIndex(index=0)])]),
})
resolution = sld_resolve(
    # Query an instance of Add for Vec2D Nat
    InstanceParam(typeclass="Add", args=[TypeNameAndArgs(name="Vec2D", args=[TypeNameAndArgs(name="Nat", args=[])])]),
    lookup_table
)

display_resolution(resolution, lookup_table)

Add Vec2D Nat -> addVec2DAdd (inst. of Add Vec2D Nat)
Add Nat -> addNat (inst. of Add Nat)


In [29]:
lookup_table = LookupTable(name_to_instance={
    # Instance that says elements of type Nat can be added together
    "addNat": TypeclassInstance(type_params=0, instance_params=[], typeclass="Add", args=[TypeNameAndArgs(name="Nat", args=[])]),
    # Instance that says elements of type Vec2D that contain things that can be added together can be added together
    "addVec2DAdd": TypeclassInstance(type_params=1, instance_params=[InstanceParam(typeclass="Add", args=[TypeParamIndex(index=0)])], typeclass="Add", args=[TypeNameAndArgs(name="Vec2D", args=[TypeParamIndex(index=0)])]),
    # Instance that says elements of type Pair that contain things that can be pairwise added together can be added together
    "addPairAddAdd": TypeclassInstance(type_params=2, instance_params=[InstanceParam(typeclass="Add", args=[TypeParamIndex(index=0)]), InstanceParam(typeclass="Add", args=[TypeParamIndex(index=1)])], typeclass="Add", args=[TypeNameAndArgs(name="Pair", args=[TypeParamIndex(index=0), TypeParamIndex(index=1)])])
})
resolution = sld_resolve(
    # Query an instance of Add for Pair (Vec2D Nat) Nat
    InstanceParam(typeclass="Add", args=[TypeNameAndArgs(name="Pair", args=[TypeNameAndArgs(name="Vec2D", args=[TypeNameAndArgs(name="Nat", args=[])]), TypeNameAndArgs(name="Nat", args=[])])]),
    lookup_table
)

display_resolution(resolution, lookup_table)

Add Pair Vec2D Nat Nat -> addPairAddAdd (inst. of Add Pair Vec2D Nat Nat)
Add Vec2D Nat -> addVec2DAdd (inst. of Add Vec2D Nat)
Add Nat -> addNat (inst. of Add Nat)
Add Nat -> addNat (inst. of Add Nat)


In [30]:
lookup_table = LookupTable(name_to_instance={
    # Instance that says A can be coerced into B
    "coeAB": TypeclassInstance(type_params=0, instance_params=[], typeclass="Coe", args=[TypeNameAndArgs(name="A", args=[]), TypeNameAndArgs(name="B", args=[])]),
    # Instance that says A can be coerced into B
    "coeBC": TypeclassInstance(type_params=0, instance_params=[], typeclass="Coe", args=[TypeNameAndArgs(name="B", args=[]), TypeNameAndArgs(name="C", args=[])]),
    # Instance that says A can be coerced into B
    "coeCD": TypeclassInstance(type_params=0, instance_params=[], typeclass="Coe", args=[TypeNameAndArgs(name="C", args=[]), TypeNameAndArgs(name="D", args=[])]),

    # Transitive closure of Coe
    "coeTCoe": TypeclassInstance(type_params=2, instance_params=[InstanceParam("Coe", args=[TypeParamIndex(index=0), TypeParamIndex(index=1)])], typeclass="CoeT", args=[TypeParamIndex(index=0), TypeParamIndex(index=1)]),
    "coeTTrans": TypeclassInstance(type_params=3, instance_params=[InstanceParam("Coe", args=[TypeParamIndex(index=0), TypeParamIndex(index=1)]), InstanceParam("CoeT", args=[TypeParamIndex(index=1), TypeParamIndex(index=2)])], typeclass="CoeT", args=[TypeParamIndex(index=0), TypeParamIndex(index=2)])
})
resolution = sld_resolve(
    # Query an instance of Coe (coercion) for A, D using the transitive closure workaround
    InstanceParam(typeclass="CoeT", args=[TypeNameAndArgs(name="A", args=[]), TypeNameAndArgs(name="D", args=[])]),
    lookup_table
)

display_resolution(resolution, lookup_table)

CoeT A D -> coeTTrans (inst. of CoeT A D)
Coe A B -> coeAB (inst. of Coe A B)
CoeT B D -> coeTTrans (inst. of CoeT B D)
Coe B C -> coeBC (inst. of Coe B C)
CoeT C D -> coeTCoe (inst. of CoeT C D)
Coe C D -> coeCD (inst. of Coe C D)


In [31]:
lookup_table = LookupTable(name_to_instance={
    # Instance that says A can be coerced into B
    "coeAB": TypeclassInstance(type_params=0, instance_params=[], typeclass="Coe", args=[TypeNameAndArgs(name="A", args=[]), TypeNameAndArgs(name="B", args=[])]),
    # Instance that says A can be coerced into B
    "coeBC": TypeclassInstance(type_params=0, instance_params=[], typeclass="Coe", args=[TypeNameAndArgs(name="B", args=[]), TypeNameAndArgs(name="C", args=[])]),
    # Instance that says A can be coerced into B
    "coeCD": TypeclassInstance(type_params=0, instance_params=[], typeclass="Coe", args=[TypeNameAndArgs(name="C", args=[]), TypeNameAndArgs(name="D", args=[])]),
    # Instance that says B can be coerced into A. Creates a cycle!
    "coeBA": TypeclassInstance(type_params=0, instance_params=[], typeclass="Coe", args=[TypeNameAndArgs(name="B", args=[]), TypeNameAndArgs(name="A", args=[])]),

    # Transitive closure of Coe
    "coeTCoe": TypeclassInstance(type_params=2, instance_params=[InstanceParam("Coe", args=[TypeParamIndex(index=0), TypeParamIndex(index=1)])], typeclass="CoeT", args=[TypeParamIndex(index=0), TypeParamIndex(index=1)]),
    "coeTTrans": TypeclassInstance(type_params=3, instance_params=[InstanceParam("Coe", args=[TypeParamIndex(index=0), TypeParamIndex(index=1)]), InstanceParam("CoeT", args=[TypeParamIndex(index=1), TypeParamIndex(index=2)])], typeclass="CoeT", args=[TypeParamIndex(index=0), TypeParamIndex(index=2)])
})
# Will get stuck in a cycle
try:
    resolution = sld_resolve(
        # Query an instance of Coe (coercion) for A, D using the transitive closure workaround
        InstanceParam(typeclass="CoeT", args=[TypeNameAndArgs(name="A", args=[]), TypeNameAndArgs(name="D", args=[])]),
        lookup_table
    )
except Exception as e:
    print(e)

Maximum depth exceeded (probably stuck in a cycle)
